In [1]:
import ee

# Reset Earth Engine completely
ee.Reset()

# Initialize with standard (normal) endpoint
ee.Initialize()

In [2]:
# Earth Engine and Common Libraries
import ee
from pathlib import Path

# Authenticate and initialize Earth Engine
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')  # Try to use existing credentials first
except Exception:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

In [3]:
# !pip install --upgrade --pre openforis-whisp



In [4]:
# Check which endpoint is now active
print("EE Data Base URL:", ee.data._cloud_api_base_url)
print("EE API Base URL:", ee.data._api_base_url)

# Check if using standard endpoint
if 'highvolume' in str(ee.data._cloud_api_base_url):
    print("❌ Still using HIGH-VOLUME endpoint")
else:
    print("✅ Now using STANDARD endpoint")

EE Data Base URL: https://earthengine-highvolume.googleapis.com
EE API Base URL: https://earthengine-highvolume.googleapis.com/api
❌ Still using HIGH-VOLUME endpoint


In [5]:

import geopandas as gpd  # for random polygon generation in tests
import random  # for random polygon generation in tests
import math  # for random polygon generation in tests
import numpy as np  # for random polygon generation in tests
from shapely.geometry import Polygon  # for random polygon generation in tests
from shapely.validation import make_valid
from shapely.geometry import mapping  # for random polygon generation in tests

def generate_random_polygon(
    min_lon, min_lat, max_lon, max_lat, min_area_ha=1, max_area_ha=10, vertex_count=20
):
    """
    Generate a random polygon within bounds with approximate area in the specified range.
    Uses a robust approach that works well with high vertex counts and never falls back to squares.

    Args:
        min_lon, min_lat, max_lon, max_lat: Boundary coordinates
        min_area_ha: Minimum area in hectares
        max_area_ha: Maximum area in hectares
        vertex_count: Number of vertices for the polygon
    """
    # Initialize variables to ensure they're always defined
    poly = None
    actual_area_ha = 0

    # Simple function to approximate area in hectares (much faster)
    def approximate_area_ha(polygon, center_lat):
        # Get area in square degrees
        area_sq_degrees = polygon.area

        # Approximate conversion factor from square degrees to hectares
        # This varies with latitude due to the Earth's curvature
        lat_factor = 111320  # meters per degree latitude (approximately)
        lon_factor = 111320 * math.cos(
            math.radians(center_lat)
        )  # meters per degree longitude

        # Convert to square meters, then to hectares (1 ha = 10,000 sq m)
        return area_sq_degrees * lat_factor * lon_factor / 10000

    # Target area in hectares
    target_area_ha = random.uniform(min_area_ha, max_area_ha)

    # Select a center point within the bounds
    center_lon = random.uniform(min_lon, max_lon)
    center_lat = random.uniform(min_lat, max_lat)

    # Initial size estimate (in degrees)
    # Rough approximation: 0.01 degrees ~ 1km at equator
    initial_radius = math.sqrt(target_area_ha / (math.pi * 100)) * 0.01

    # Avoid generating too many points initially - cap vertex count for stability
    effective_vertex_count = min(
        vertex_count, 100
    )  # Cap at 100 to avoid performance issues

    # Primary approach: Create polygon using convex hull approach
    for attempt in range(5):  # First method gets 5 attempts
        try:
            # Generate random points in a circle around center with varying distance
            thetas = np.linspace(0, 2 * math.pi, effective_vertex_count, endpoint=False)

            # Add randomness to angles - smaller randomness for higher vertex counts
            angle_randomness = min(0.2, 2.0 / effective_vertex_count)
            thetas += np.random.uniform(
                -angle_randomness, angle_randomness, size=effective_vertex_count
            )

            # Randomize distances from center - less extreme for high vertex counts
            distance_factor = min(0.3, 3.0 / effective_vertex_count) + 0.7
            distances = initial_radius * np.random.uniform(
                1.0 - distance_factor / 2,
                1.0 + distance_factor / 2,
                size=effective_vertex_count,
            )

            # Convert to cartesian coordinates
            xs = center_lon + distances * np.cos(thetas)
            ys = center_lat + distances * np.sin(thetas)

            # Ensure points are within bounds
            xs = np.clip(xs, min_lon, max_lon)
            ys = np.clip(ys, min_lat, max_lat)

            # Create vertices list
            vertices = list(zip(xs, ys))

            # Close the polygon
            if vertices[0] != vertices[-1]:
                vertices.append(vertices[0])

            # Create polygon
            poly = Polygon(vertices)

            # Ensure it's valid
            if not poly.is_valid:
                poly = make_valid(poly)
                if poly.geom_type != "Polygon":
                    # If not a valid polygon, we'll try again
                    continue

            # Calculate approximate area
            actual_area_ha = approximate_area_ha(poly, center_lat)

            # Check if within target range
            if min_area_ha * 0.8 <= actual_area_ha <= max_area_ha * 1.2:
                return poly, actual_area_ha

            # Adjust size for next attempt based on ratio
            if actual_area_ha > 0:  # Avoid division by zero
                scale_factor = math.sqrt(target_area_ha / actual_area_ha)
                initial_radius *= scale_factor

        except Exception as e:
            print(f"Error in convex hull method (attempt {attempt+1}): {e}")

    # Second approach: Star-like pattern with controlled randomness
    # This is a fallback that will still create an irregular polygon, not a square
    for attempt in range(5):  # Second method gets 5 attempts
        try:
            # Use fewer vertices for stability in the fallback
            star_vertex_count = min(15, vertex_count)
            vertices = []

            # Create a star-like pattern with two radiuses
            for i in range(star_vertex_count):
                angle = 2 * math.pi * i / star_vertex_count

                # Alternate between two distances to create star-like shape
                if i % 2 == 0:
                    distance = initial_radius * random.uniform(0.7, 0.9)
                else:
                    distance = initial_radius * random.uniform(0.5, 0.6)

                # Add some irregularity to angles
                angle += random.uniform(-0.1, 0.1)

                # Calculate vertex position
                lon = center_lon + distance * math.cos(angle)
                lat = center_lat + distance * math.sin(angle)

                # Ensure within bounds
                lon = min(max(lon, min_lon), max_lon)
                lat = min(max(lat, min_lat), max_lat)

                vertices.append((lon, lat))

            # Close the polygon
            vertices.append(vertices[0])

            # Create polygon
            poly = Polygon(vertices)
            if not poly.is_valid:
                poly = make_valid(poly)
                if poly.geom_type != "Polygon":
                    continue

            actual_area_ha = approximate_area_ha(poly, center_lat)

            # We're less picky about size at this point, just return it
            if actual_area_ha > 0:
                return poly, actual_area_ha

            # Still try to adjust if we get another attempt
            if actual_area_ha > 0:
                scale_factor = math.sqrt(target_area_ha / actual_area_ha)
                initial_radius *= scale_factor

        except Exception as e:
            print(f"Error in star pattern method (attempt {attempt+1}): {e}")

    # Last resort - create a perturbed circle (never a square)
    try:
        # Create a circle-like shape with small perturbations
        final_vertices = []
        perturbed_vertex_count = 8  # Use a modest number for stability

        for i in range(perturbed_vertex_count):
            angle = 2 * math.pi * i / perturbed_vertex_count
            # Small perturbation
            distance = initial_radius * random.uniform(0.95, 1.05)

            # Calculate vertex position
            lon = center_lon + distance * math.cos(angle)
            lat = center_lat + distance * math.sin(angle)

            # Ensure within bounds
            lon = min(max(lon, min_lon), max_lon)
            lat = min(max(lat, min_lat), max_lat)

            final_vertices.append((lon, lat))

        # Close the polygon
        final_vertices.append(final_vertices[0])

        # Create polygon
        poly = Polygon(final_vertices)
        if not poly.is_valid:
            poly = make_valid(poly)

        actual_area_ha = approximate_area_ha(poly, center_lat)

    except Exception as e:
        print(f"Error in final fallback method: {e}")
        # If absolutely everything fails, create the simplest valid polygon (triangle)
        # This is different from a square and should be more compatible with your code
        offset = initial_radius / 2
        poly = Polygon(
            [
                (center_lon, center_lat + offset),
                (center_lon + offset, center_lat - offset),
                (center_lon - offset, center_lat - offset),
                (center_lon, center_lat + offset),
            ]
        )
        actual_area_ha = approximate_area_ha(poly, center_lat)

    # Return whatever we've created - never a simple square
    return poly, actual_area_ha


def generate_properties(area_ha, index):
    """
    Generate properties for features with sequential internal_id

    Args:
        area_ha: Area in hectares of the polygon
        index: Index of the feature to use for sequential ID
    """
    return {
        "internal_id": index + 1,  # Create sequential IDs starting from 1
    }


def create_geojson(
    bounds,
    num_polygons=25,
    min_area_ha=1,
    max_area_ha=10,
    min_number_vert=10,
    max_number_vert=20,
):
    """Create a GeoJSON file with random polygons within area range"""
    min_lon, min_lat, max_lon, max_lat = bounds
    # min_number_vert = 15
    # max_number_vert = 20

    features = []
    for i in range(num_polygons):
        # Random vertex count between 4 and 8
        # vertices = random.randint(4, 8)
        vertices = random.randint(min_number_vert, max_number_vert)

        # Generate polygon with area control
        polygon, actual_area = generate_random_polygon(
            min_lon,
            min_lat,
            max_lon,
            max_lat,
            min_area_ha=min_area_ha,
            max_area_ha=max_area_ha,
            vertex_count=vertices,
        )

        # Create GeoJSON feature with actual area
        properties = generate_properties(actual_area, index=i)
        feature = {
            "type": "Feature",
            "properties": properties,
            "geometry": mapping(polygon),
        }

        features.append(feature)

    # Create the GeoJSON feature collection
    geojson = {"type": "FeatureCollection", "features": features}

    return geojson


def reformat_geojson_properties(
    geojson_path,
    output_path=None,
    id_field="internal_id",
    start_index=1,
    remove_properties=False,
    add_uuid=False,
):
    """
    Add numeric IDs to features in an existing GeoJSON file and optionally remove properties.

    Args:
        geojson_path: Path to input GeoJSON file
        output_path: Path to save the output GeoJSON (if None, overwrites input)
        id_field: Name of the ID field to add
        start_index: Starting index for sequential IDs
        remove_properties: Whether to remove all existing properties (default: False)
        add_uuid: Whether to also add UUID field

    Returns:
        GeoDataFrame with updated features
    """

    # Read the GeoJSON
    # print(f"Reading GeoJSON file: {geojson_path}")
    gdf = gpd.read_file(geojson_path)

    # Remove existing properties if requested
    if remove_properties:
        # Keep only the geometry column and drop all other columns
        gdf = gdf[["geometry"]].copy()
        # print(f"Removed all existing properties from features")

    # Add sequential numeric IDs
    gdf[id_field] = [i + start_index for i in range(len(gdf))]

    # Optionally add UUIDs
    if add_uuid:
        gdf["uuid"] = [str(uuid.uuid4()) for _ in range(len(gdf))]

    # Write the GeoJSON with added IDs
    output_path = output_path or geojson_path
    gdf.to_file(output_path, driver="GeoJSON")
    print(f"Added {id_field} to GeoJSON and saved to {output_path}")

    return None


In [6]:
import openforis_whisp as whisp



In [7]:

whisp_image = whisp.combine_datasets()

Whisp multiband image compiled


In [8]:
import ee
import geopandas as gpd
import pandas as pd
import time
import threading
from queue import Queue
import logging
from typing import List, Optional, Dict, Any
from concurrent.futures import ThreadPoolExecutor, as_completed

# Simplified logging setup
logging.basicConfig(level=logging.WARNING)  # Reduce default verbosity
logger = logging.getLogger("whisp-batch")

# # defaults
# Optimized configuration for EE high-volume processing
EE_MAX_CONCURRENT = 10
EE_FEATURES_PER_BATCH = 50  # Features per Earth Engine request
MAX_RETRIES = 3
THREAD_POOL_SIZE = 4

class OptimizedWhispProcessor:
    """Optimized processor using Earth Engine high-volume patterns"""
    
    def __init__(self, max_concurrent=EE_MAX_CONCURRENT, features_per_batch=EE_FEATURES_PER_BATCH):
        self.max_concurrent = max_concurrent
        self.features_per_batch = features_per_batch
        self.semaphore = threading.Semaphore(max_concurrent)
        self.results = {}
        self.processing_stats = {'completed': 0, 'failed': 0, 'total': 0}
        
    def process_file_optimized(self, geojson_path: str, national_codes: Optional[List[str]] = None) -> pd.DataFrame:
        """Process file using optimized Earth Engine batching"""
        
        # Load the GeoDataFrame
        gdf = gpd.read_file(geojson_path)
        total_features = len(gdf)
        
        # Split into feature batches
        feature_batches = []
        for i in range(0, total_features, self.features_per_batch):
            batch = gdf.iloc[i:i+self.features_per_batch]
            feature_batches.append(batch)
        
        total_batches = len(feature_batches)
        print(f"📊 Processing {total_features:,} features in {total_batches} batches ({self.features_per_batch} features/batch)")
        print(f"🔄 Running {self.max_concurrent} concurrent requests...")
        
        # Track progress
        completed_batches = 0
        failed_batches = 0
        
        # Process batches concurrently using ThreadPoolExecutor
        results = []
        with ThreadPoolExecutor(max_workers=self.max_concurrent) as executor:
            # Submit all batches
            future_to_batch = {
                executor.submit(self._process_feature_batch, batch, national_codes, i): i 
                for i, batch in enumerate(feature_batches)
            }
            
            # Collect results as they complete
            for future in as_completed(future_to_batch):
                batch_idx = future_to_batch[future]
                try:
                    batch_result = future.result()
                    results.append(batch_result)
                    completed_batches += 1
                    
                    # Show progress every 10 batches or at completion
                    if completed_batches % 10 == 0 or completed_batches == total_batches:
                        print(f"✅ Completed: {completed_batches}/{total_batches} batches ({completed_batches/total_batches*100:.0f}%)")
                        
                except Exception as e:
                    failed_batches += 1
                    print(f"❌ Batch {batch_idx + 1} failed: {str(e)[:50]}...")
                    self.processing_stats['failed'] += 1
        
        # Final summary
        if results:
            combined_df = pd.concat(results, ignore_index=True)
            print(f"🎉 Successfully processed {len(combined_df):,} features!")
            if failed_batches > 0:
                print(f"⚠️  {failed_batches} batches failed")
            return combined_df
        else:
            print("❌ No results produced")
            return pd.DataFrame()
    
    def _process_feature_batch(self, batch_gdf: gpd.GeoDataFrame, national_codes: Optional[List[str]], batch_idx: int) -> pd.DataFrame:
        """Process a single batch of features using Earth Engine"""
        
        with self.semaphore:  # Limit concurrent EE requests
            # Convert GeoDataFrame to Earth Engine FeatureCollection directly
            features_list = []
            
            for idx, row in batch_gdf.iterrows():
                # Convert each feature to EE format
                geometry = row.geometry
                properties = {k: v for k, v in row.items() if k != 'geometry' and pd.notna(v)}
                
                # Convert shapely geometry to EE geometry
                if geometry is not None:
                    ee_geometry = ee.Geometry(geometry.__geo_interface__)
                    ee_feature = ee.Feature(ee_geometry, properties)
                    features_list.append(ee_feature)
            
            # Create FeatureCollection from features
            feature_collection = ee.FeatureCollection(features_list)
            
            # Process using optimized Earth Engine parameters
            return self._process_ee_feature_collection(feature_collection, national_codes, batch_idx)

    def _process_ee_feature_collection(self, feature_collection: ee.FeatureCollection, 
                                 national_codes: Optional[List[str]], batch_idx: int) -> pd.DataFrame:
        """Process FeatureCollection using Earth Engine with retry logic"""
        
        for attempt in range(MAX_RETRIES):
            try:
                # Use whisp's existing function that handles all conversions
                df_result = whisp.whisp_stats_ee_to_df(
                    feature_collection=feature_collection,
                    external_id_column=None,
                    remove_geom=False,
                    national_codes=national_codes,
                    unit_type="ha",
                    whisp_image=whisp_image,
                )
                
                return df_result
                
            except ee.EEException as e:
                if attempt < MAX_RETRIES - 1:
                    backoff = min(2 ** attempt, 30)
                    time.sleep(backoff)
                else:
                    raise e
            except Exception as e:
                if attempt < MAX_RETRIES - 1:
                    time.sleep(2 ** attempt)
                else:
                    raise e
        
        raise RuntimeError(f"Failed to process batch {batch_idx + 1} after {MAX_RETRIES} attempts")

# # Example usage with controlled batch sizes
# if __name__ == "__main__":
    
#     # Configure batch size based on your data characteristics
#     FEATURES_PER_EE_REQUEST = 25  # Small batches for complex geometries
#     MAX_CONCURRENT_EE_REQUESTS = 20  # Conservative for quota management
    
#     # Initialize processor
#     processor = OptimizedWhispProcessor(
#         max_concurrent=MAX_CONCURRENT_EE_REQUESTS,
#         features_per_batch=FEATURES_PER_EE_REQUEST
#     )
    
#     # Process file with controlled batching
#     try:
#         result_df = processor.process_file_optimized(
#             GEOJSON_EXAMPLE_FILEPATH, 
#             national_codes=["br", "co"]
#         )
        
#         if not result_df.empty:
#             print(f"\n📄 Saving results to CSV...")
#             result_df.to_csv("optimized_whisp_results.csv", index=False)
#             print(f"💾 Results saved to optimized_whisp_results.csv")
            
#             print(f"\n📋 First 5 rows:")
#             print(result_df.head())
#         else:
#             print("❌ No results to save")
            
#     except Exception as e:
#         print(f"💥 Processing failed: {e}")

In [9]:
!pip show openforis-whisp

Name: openforis-whisp
Version: 2.0.0b1
Summary: Whisp (What is in that plot) is an open-source solution which helps to produce relevant forest monitoring information and support compliance with deforestation-related regulations.
Home-page: 
Author: Andy Arnell
Author-email: andrew.arnell@fao.org
License: MIT
Location: c:\Users\Arnell\Documents\GitHub\whisp\.venv\Lib\site-packages
Editable project location: C:\Users\Arnell\Documents\GitHub\whisp
Requires: country_converter, earthengine-api, geojson, geopandas, ipykernel, numpy, pandas, pandera, pydantic-core, python-dotenv, rsa, shapely
Required-by: 


In [10]:
folder_path = (r"C:\Users\Arnell\Downloads\processing_tests")  # Replace with your folder path

In [11]:
GEOJSON_EXAMPLE_FILEPATH = folder_path+"/random_polygons.geojson"

# Define bounds from the provided Earth Engine geometry
# # area in Ghana 
# bounds = [ 
#     -3.04548260909834,  # min_lon
#     5.253961384163733,  # min_lat
#     -1.0179939534016594,  # max_lon
#     7.48307210714245    # max_lat
# ]

# area in China
bounds = [
    103.44831497309737,  # min_lon
    25.686366665187148,  # min_lat
    109.57868606684737,  # max_lon
    28.79200348254393    # max_lat
]

In [12]:
# random_geojson = whisp.create_geojson(
random_geojson = create_geojson(
    bounds, 
    num_polygons=500, 
    min_area_ha=5, 
    max_area_ha=10, 
    min_number_vert=90, 
    max_number_vert=100)

GEOJSON_EXAMPLE_FILEPATH = folder_path + "/random_polygons.geojson"

import json
# Save the GeoJSON to a file
with open(GEOJSON_EXAMPLE_FILEPATH, 'w') as f:
    json.dump(random_geojson, f)

# Use example Whisp inputs (optional)
# GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path("geojson_example.geojson")


# Add IDs to your existing GeoJSON file

# #Save to a new file (instead of overwriting)
# # whisp.reformat_geojson_properties(
# whisp.reformat_geojson_properties(
    
#     geojson_path=GEOJSON_EXAMPLE_FILEPATH, 
#     id_field="internal_id",
#     output_path=folder_path + "/random_polygons_with_ids.geojson",
#     remove_properties=True
# )

In [14]:

# Example usage with controlled batch sizes
if __name__ == "__main__":
    
    # Configure batch size based on your data characteristics
    FEATURES_PER_EE_REQUEST = 25  # Small batches for complex geometries
    MAX_CONCURRENT_EE_REQUESTS = 20  # Conservative for quota management
    
    # Initialize processor
    processor = OptimizedWhispProcessor(
        max_concurrent=MAX_CONCURRENT_EE_REQUESTS,
        features_per_batch=FEATURES_PER_EE_REQUEST
    )
    
    # Process file with controlled batching
    try:
        # GEOJSON_EXAMPLE_FILEPATH = whisp.get_example_data_path("geojson_example.geojson")
        
        logger.info(f"Processing with {FEATURES_PER_EE_REQUEST} features per Earth Engine request")
        logger.info(f"Maximum {MAX_CONCURRENT_EE_REQUESTS} concurrent requests")
        
        result_df = processor.process_file_optimized(
            GEOJSON_EXAMPLE_FILEPATH, 
            # national_codes=["br", "co"]
        )
        
        if not result_df.empty:
            print(f"Success! Processed {len(result_df)} features")
            print("\nFirst 5 rows:")
            print(result_df.head())
            
            # Save results
            result_df.to_csv("optimized_whisp_results.csv", index=False)
            logger.info("Results saved to optimized_whisp_results.csv")
        else:
            print("No results produced")
            
        print(f"Processing stats: {processor.processing_stats}")
        
    except Exception as e:
        logger.error(f"Processing failed: {e}")

2025-10-12 10:47:41,423 - INFO - Processing with 25 features per Earth Engine request
2025-10-12 10:47:41,424 - INFO - Maximum 20 concurrent requests


📊 Processing 500 features in 20 batches (25 features/batch)
🔄 Running 20 concurrent requests...


2025-10-12 10:47:52,280 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10
2025-10-12 10:47:52,318 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10
2025-10-12 10:47:52,333 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10
2025-10-12 10:47:52,578 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10
2025-10-12 10:47:52,849 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10
2025-10-12 10:47:53,129 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googleapis.com. Connection pool size: 10
2025-10-12 10:47:53,129 - WARNING - Connection pool is full, discarding connection: earthengine-highvolume.googl

✅ Completed: 10/20 batches (50%)


2025-10-12 10:48:11,982 - INFO - Results saved to optimized_whisp_results.csv


✅ Completed: 20/20 batches (100%)
🎉 Successfully processed 500 features!
Success! Processed 500 features

First 5 rows:
                                                 geo     Admin_Level_1   Area  \
0  {'type': 'Polygon', 'coordinates': [[[104.1493...   Yunnan Province  9.842   
1  {'type': 'Polygon', 'coordinates': [[[103.5543...   Yunnan Province  7.038   
2  {'type': 'Polygon', 'coordinates': [[[108.4779...  Guizhou Province  5.881   
3  {'type': 'Polygon', 'coordinates': [[[103.7600...   Yunnan Province  8.254   
4  {'type': 'Polygon', 'coordinates': [[[107.7283...  Guizhou Province  6.436   

   Centroid_lat  Centroid_lon  Cocoa_2023_FDaP  Cocoa_ETH  Cocoa_FDaP  \
0     26.224426    104.151540                0          0           0   
1     27.029586    103.556195                0          0           0   
2     27.984908    108.479651                0          0           0   
3     26.781161    103.762100                0          0           0   
4     27.514257    107.73013

In [7]:
result_df  # Display first few rows of combined results

""


Classic Whisp

In [15]:
# Earth Engine and Common Libraries
import ee
from pathlib import Path

# Authenticate and initialize Earth Engine
try:
    ee.Initialize()  # Try to use existing credentials first
except Exception:
    ee.Authenticate()
    ee.Initialize()

In [16]:
import ee

# Reset Earth Engine completely
ee.Reset()

# Initialize with standard (normal) endpoint
ee.Initialize()

print("✅ Switched to STANDARD Earth Engine endpoint")

# Check the data module's base URL
print("EE Data Base URL:", ee.data._cloud_api_base_url)
print("EE API Base URL:", ee.data._api_base_url)

# Check if high-volume endpoint is configured
if 'highvolume' in str(ee.data._cloud_api_base_url):
    print("Using HIGH-VOLUME endpoint")
else:
    print("Using STANDARD endpoint")

✅ Switched to STANDARD Earth Engine endpoint
EE Data Base URL: https://earthengine.googleapis.com
EE API Base URL: https://earthengine.googleapis.com/api
Using STANDARD endpoint


In [17]:
import openforis_whisp as whisp


In [18]:
!pip show openforis-whisp

Name: openforis-whisp
Version: 2.0.0b1
Summary: Whisp (What is in that plot) is an open-source solution which helps to produce relevant forest monitoring information and support compliance with deforestation-related regulations.
Home-page: 
Author: Andy Arnell
Author-email: andrew.arnell@fao.org
License: MIT
Location: c:\Users\Arnell\Documents\GitHub\whisp\.venv\Lib\site-packages
Editable project location: C:\Users\Arnell\Documents\GitHub\whisp
Requires: country_converter, earthengine-api, geojson, geopandas, ipykernel, numpy, pandas, pandera, pydantic-core, python-dotenv, rsa, shapely
Required-by: 


In [19]:
# whisp = whisp.whisp_formatted_stats_geojson_to_df(GEOJSON_EXAMPLE_FILEPATH)
whisp = whisp.whisp_stats_geojson_to_df(GEOJSON_EXAMPLE_FILEPATH,whisp_image=whisp_image)

Reading GeoJSON file from: C:\Users\Arnell\Downloads\processing_tests\random_polygons.geojson
